# Init

In [1]:
from IPython.display import display
from typing import List,Dict
import os
import sys
import psycopg2
from psycopg2.extras import NamedTupleCursor

os.chdir('/home/rchuzh/programming/image_labelling_shrdc')
from pathlib import Path
Path.cwd()
SRC = Path('/home/rchuzh/programming/image_labelling_shrdc/src')  # ROOT folder -> ./src
LIB_PATH = SRC / "lib"

if str(LIB_PATH) not in sys.path:
    sys.path.insert(0, str(LIB_PATH))  # ./lib
else:
    pass

In [2]:
from data_manager.database_manager import db_fetchone,db_no_fetch,db_fetchall
from core.utils.log import log_info,log_error
dsn = "host=localhost port=5432 dbname=eye user=shrdc password=shrdc"
def init_connection(dsn=None, connection_factory=None, cursor_factory=None, **kwargs):
    """ Connect to the PostgreSQL database server """

    try:
        # read connection parameters
        # params = config()

        # connect to the PostgreSQL server
        log_info('Connecting to the PostgreSQL database...')
        if kwargs:
            conn = psycopg2.connect(**kwargs)

        else:
            conn = psycopg2.connect(dsn, connection_factory, cursor_factory)

        # create a cursor
        with conn:
            with conn.cursor(cursor_factory=NamedTupleCursor) as cur:

                # execute a statement
                cur.execute('SELECT version();')
                conn.commit()

                # display the PostgreSQL database server version
                db_version = cur.fetchone().version
                log_info(f"PostgreSQL database version: {db_version}")
                log_info(f"PostgreSQL connection status: {conn.info.status}")
                log_info(
                    f"You are connected to database '{conn.info.dbname}' as user '{conn.info.user}' on host '{conn.info.host}' at port '{conn.info.port}'.")
        return conn
    except (Exception, psycopg2.DatabaseError) as error:
        log_error(error)
        conn = None


In [3]:
conn=init_connection(dsn)

2021-08-06 12:24:18.273 INFO    root: Connecting to the PostgreSQL database...
2021-08-06 12:24:18.306 INFO    root: PostgreSQL database version: PostgreSQL 13.3 (Ubuntu 13.3-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit
2021-08-06 12:24:18.308 INFO    root: PostgreSQL connection status: 0
2021-08-06 12:24:18.312 INFO    root: You are connected to database 'eye' as user 'shrdc' on host 'localhost' at port '5432'.


In [4]:
from typing import Union,List,Dict,Optional
import pandas as pd
def create_dataframe(data: Union[List, Dict, pd.Series], column_names: List = None, sort: bool = False, sort_by: Optional[str] = None, asc:bool=True,date_time_format: bool = False) -> pd.DataFrame:
    if data:

        df = pd.DataFrame(data, columns=column_names)
        df.index.name = ('No.')
        if date_time_format:
            df['Date/Time'] = pd.to_datetime(df['Date/Time'],
                                             format='%Y-%m-%d %H:%M:%S')

            # df.sort_values(by=['Date/Time'], inplace=True,
            #                ascending=False, ignore_index=True)
        if sort:

            df.sort_values(by=[sort_by], inplace=True,
                           ascending=asc, ignore_index=True)

            # dfStyler = df.style.set_properties(**{'text-align': 'center'})
            # dfStyler.set_table_styles(
            #     [dict(selector='th', props=[('text-align', 'center')])])

        return df

# 2/8/2021

In [54]:
query_sql="""
SELECT
            id AS "ID",
            name AS "Name",
            dataset_size AS "Dataset Size",
              (SELECT ft.name AS "Filetype" from public.filetype ft where ft.id = d.filetype_id),
            updated_at AS "Date/Time",
            description AS "Description",
            dataset_path AS "Dataset Path"
          
        FROM
            public.dataset d;
"""

In [55]:
query,column_names=db_fetchall(query_sql,conn,fetch_col_name=True,return_dict=False)



2021-08-06 14:21:09.337 ERROR   root: relation "public.file_type" does not exist
LINE 6:             (SELECT ft.name AS "File Type" from public.file_...
                                                        ^



TypeError: cannot unpack non-iterable NoneType object

In [ ]:
[dict(row) for row in query]

In [7]:
selection=[x.ID for x in query if x.Name=="My Third Dataset"][0]
selection

3

In [8]:
dataset_tmp = []
if query:
    for dataset in query:

        converted_datetime = dataset.Date_Time.strftime(
            '%Y-%m-%d %H:%M:%S')

        # convert datetime with TZ to (2021-07-30 12:12:12) format
        dataset = dataset._replace(
            Date_Time=converted_datetime)
        dataset_tmp.append(dataset)

In [9]:
df=create_dataframe(dataset_tmp,column_names=column_names,sort=True,sort_by='ID',asc=True,date_time_format=True)

In [10]:
df_loc=df.loc[:,"ID":"Date/Time"]
df_loc
df

,ID,Name,Dataset Size,Filetype,Date/Time,Description,Dataset Path
0,2,My Second Dataset,4,Image,2021-08-03 21:49:23,None,/home/rchuzh/.local/share/integrated-vision-in...
1,3,My Third Dataset,4,Image,2021-08-03 21:49:23,None,/home/rchuzh/.local/share/integrated-vision-in...
2,4,My Fourth Dataset,4,Image,2021-08-03 21:49:23,asdadsfads,/home/rchuzh/.local/share/integrated-vision-in...


In [75]:
df_dict=df.to_dict(orient='index')

In [78]:
if df_dict.get(3):
    print("yes")
else:
    print("no")

no


In [56]:
name=(df_loc[df_loc["ID"]==3].Name)[1]
name

'My Third Dataset'

In [63]:
df_loc.loc[df_loc["ID"]==3,'Name'].values[0]

'My Third Dataset'

In [79]:
some_dict={'one':[{'name':'hello','info':'sadfa'},{'name':'test','info':'jvhjfgh'}]}

In [84]:
multi_dict={}
for outerkey,innerDict in some_dict.items():
    for innerkey,value in some_dict.items():
        multi_dict[(outerkey,innerkey)]=value

In [86]:
pd.DataFrame.from_dict(multi_dict)

,one
,one
0,"{'name': 'hello', 'info': 'sadfa'}"
1,"{'name': 'test', 'info': 'jvhjfgh'}"


In [ ]:
test=[1,2,3,4,5]
for z in zip(test[:],test[1:]):
    # if x==y:
    #     pass
    # else:
    #     print("Not equal")
    for i in z:
        print(i)

In [18]:
# MIME: type/subtype
#get filetype 
import mimetypes
from pathlib import Path
from streamlit.uploaded_file_manager import UploadedFile
filepath=Path('/home/rchuzh/Documents/aruco/train/IMG_20210315_181136.jpg'
)
def get_filetype(file:Union[str,Path,UploadedFile]):
    if isinstance(file,(str,Path)):
        mime_type=mimetypes.guess_type(file)[0]
        filetype=str(Path(mime_type).parent)
    elif isinstance(file,UploadedFile):
        mime_type=file.type
        
    filetype=str(Path(mime_type).parent)
    return filetype

In [11]:
file_tuple=[Path('/home/rchuzh/Documents/aruco/train/IMG_20210315_181136.jpg'
),Path('/home/rchuzh/Documents/aruco/test/IMG_20210315_181130.jpg')]
filetype1,filetype2=list(map(get_filetype,file_tuple))

NameError: name 'get_filetype' is not defined

In [26]:
filetype1,filetype2

('image', 'image')

In [8]:
# compare 2 values
def compare_filetypes(file_tuple:tuple):
    filetype1,filetype2=list(map(get_filetype,file_tuple))
    
    print(file_tuple)
    if filetype1==filetype2:
        return True
    else:
        
        return False

In [9]:
uploaded_files=[1,2,3,4,4]


filetypes=map(compare_filetypes,zip(uploaded_files[:],uploaded_files[1:]))
for filetype in filetypes:
    if filetype:
        log_info("Filetype passed")
        pass
    else:
        log_error("Filetype different")
        break

2021-08-04 09:20:29.278 ERROR   root: Filetype different


(1, 2)


In [18]:
# fileuploader consist of more tha one file upload
# if one file then no execution of callback
def check_filetype_category(uploaded_files:Union[str,Path,UploadedFile]):
    if uploaded_files:
        if len(uploaded_files)==1:
            filetype=get_filetype(uploaded_files)

        else:
            filetypes=map(compare_filetypes,zip(uploaded_files[:],uploaded_files[1:]))
            for filetype in filetypes:
                if filetype:
                    log_info("Filetype passed")
                    pass
                else:
                    log_error("Filetype different")
                    break



SyntaxError: unexpected EOF while parsing (<ipython-input-18-d4711ecc504b>, line 4)

In [14]:
numbers1=[1,2,3,4]
numbers2=[5,6,7,8]
number=Path('3')
result=map(lambda x,y:x+y,numbers1,numbers2) #iterator
result=list(result)
result
isinstance(number,Path)

True

In [16]:
number=[3]
len(number)

1

# 4/8/2021

In [1]:
def trace(f):
    def wrap(*args,**kwargs):
        print(f"[TRACE] func: {f.__name__}, args: {args}, kwargs: {kwargs}")
        return f(*args,**kwargs)
    return wrap

In [2]:
@trace
def add_two(x):
    return x+2

In [3]:
add_two(3)

[TRACE] func: add_two, args: (3,), kwargs: {}


5

In [10]:
(trace(lambda x:x**2))(3)

[TRACE] func: <lambda>, args: (3,), kwargs: {}


9

In [5]:
add_two=lambda x:x**2

In [9]:
print(trace(add_two)(3))

[TRACE] func: <lambda>, args: (3,), kwargs: {}
9


In [35]:
14%5

4

In [38]:
import os
stat=os.stat('/home/rchuzh/Documents/aruco/train/IMG_20210315_181136.jpg')

In [39]:
stat

os.stat_result(st_mode=33204, st_ino=4862438, st_dev=66309, st_nlink=1, st_uid=1000, st_gid=1000, st_size=1038614, st_atime=1628184478, st_mtime=1616384801, st_ctime=1617272005)

In [13]:
from time import ctime
import time
ctime(stat.st_mtime)    #local time
time.strftime('%Y-%m-%d', time.localtime(stat.st_mtime))

'2021-03-22'

In [40]:
from datetime import datetime
mtime=datetime.fromtimestamp(stat.st_mtime).strftime('%Y-%m-%d')
# '%Y-%m-%d %H:%M:%S'


In [41]:
print(mtime)

2021-03-22


In [45]:
import os
from glob import iglob,glob
from datetime import datetime
def get_data_name_list(name,dataset_path, data_name_list_full: Dict = {}):
    """Obtain list of data in the dataset 
        - Iterative glob through the dataset directory
        - Obtain filename using pathlib.Path(<'filepath/*'>).name

    Returns:
        Dict[dict]: Dataset name as key to a List of data in the dataset directory
    """
    # IF dataset info already exist -> get from Dict
    if data_name_list_full.get(name) and (len(data_name_list_full.get(name))) == len([file for file in Path(dataset_path).iterdir() if file.is_file()]):

        data_name_list = data_name_list_full.get(name)
        log_info("Enter top")
        return data_name_list

    else:
        if dataset_path:
            log_info("Enter bottom")

            dataset_path = dataset_path + "/*"
            log_info(dataset_path)
            
# i need    
# {'id':data_name,'filetype':self.filetype,'created_at':os.stat().st_mtime}
            data_info = {}
            data_info_tmp=[]
            # Glob through dataset directory
            for data_path in iglob(dataset_path):
                
                log_info(f"Globbing {data_path}......")
                data_info['id'] = Path(data_path).name
                data_info['filetype'] ='Image'

                # Get File Modified Time
                data_modified_time_epoch = os.stat(str(data_path)).st_mtime
                data_modified_time = datetime.fromtimestamp(data_modified_time_epoch
                                                            ).strftime('%Y-%m-%d')
                data_info['created'] = data_modified_time
                data_info_tmp.append(data_info)
            data_name_list_full[name] = data_info_tmp

        return data_name_list_full


In [44]:
dataset_info=df.loc[1,:]
dataset_info_dict=dataset_info.to_dict()
dataset_info_dict

{'ID': 3,
 'Name': 'My Third Dataset',
 'Dataset Size': 4,
 'Filetype': 'Image',
 'Date/Time': Timestamp('2021-08-03 21:49:23'),
 'Description': None,
 'Dataset Path': '/home/rchuzh/.local/share/integrated-vision-inspection-system/app_media/dataset/My-Third-Dataset'}

In [48]:
data_name_list=get_data_name_list(dataset_info_dict['Name'],dataset_info_dict['Dataset Path'])

2021-08-06 13:00:43.871 INFO    root: Enter top


In [49]:
data_name_list

[{'id': 'IMG_20210315_184236.jpg',
  'filetype': 'Image',
  'created': '2021-07-20'},
 {'id': 'IMG_20210315_184236.jpg',
  'filetype': 'Image',
  'created': '2021-07-20'},
 {'id': 'IMG_20210315_184236.jpg',
  'filetype': 'Image',
  'created': '2021-07-20'},
 {'id': 'IMG_20210315_184236.jpg',
  'filetype': 'Image',
  'created': '2021-07-20'}]

In [51]:
file_list=[file for file in Path(dataset_info_dict['Dataset Path']).iterdir() if file.is_file()]

In [53]:
len(file_list) == len(data_name_list)

True